In [1]:
from adalflow.components.model_client import OpenAIClient
from dotenv import load_dotenv

from dystopic_investment_aigents.agents.base_agents.newsletter_base import NewsletterAdal


load_dotenv()

newsletter_agent = NewsletterAdal(
    personality={"mood": "optimistic", "risk_tolerance": 0.5},
    seniority=OpenAIClient(),
    seniority_args= {
        "model": "gpt-4o-mini",
        "temperature": 0.5,
    }
)

In [2]:
import os
from sqlalchemy import create_engine
import pandas as pd

from dystopic_investment_aigents.data_ingestion.db.postgres_db import PostgresConfig
from dystopic_investment_aigents.agents.base_agents.newsletter_base import Newsletter
from dystopic_investment_aigents.agents.base_agents.quant_trader_base import Asset
from dystopic_investment_aigents.agents.base_agents.quant_trader_base import Portfolio

db_uri = PostgresConfig(
    host=os.environ["SB_DDBB_HOST"],
    port=os.environ["SB_DDBB_PORT"],
    database=os.environ["SB_DDBB_DATABASE"],
    user=os.environ["SB_DDBB_USER"],
    password=os.environ["SB_DDBB_PWD"]
).get_connection_string()

engine = create_engine(db_uri)

df_current_portfolio = pd.read_sql("SELECT * FROM portfolio WHERE created_at = (SELECT max(created_at) FROM portfolio)", engine)

In [3]:
df_current_portfolio.head(2)

,asset_name,asset_type,weight,created_at,asset_short_name,portfolio_trader
0,IBM,stock,0.15,2024-08-05 12:08:41.332903+00:00,IBM,QuantTraderNaiveAdal
1,Palantir,stock,0.20,2024-08-05 12:08:41.332903+00:00,PLTR,QuantTraderNaiveAdal


In [4]:
current_portfolio_assets = [Asset(name=n, type=t) for n, t in  zip(df_current_portfolio["asset_name"], df_current_portfolio["asset_type"])]
current_portfolio = Portfolio(allocation=dict(zip(current_portfolio_assets, df_current_portfolio["weight"])))
current_portfolio

Portfolio(allocation={Asset(name='IBM', type='stock'): 0.15, Asset(name='Palantir', type='stock'): 0.2, Asset(name='Ford Motor Company', type='stock'): 0.05, Asset(name='Oatly Group AB', type='stock'): 0.05, Asset(name='Corteva, Inc.', type='stock'): 0.05, Asset(name='VeriSign, Inc.', type='stock'): 0.05, Asset(name='Adyen N.V.', type='stock'): 0.05, Asset(name='Beyond Meat, Inc.', type='stock'): 0.05, Asset(name='Altria Group, Inc.', type='stock'): 0.03, Asset(name='The Mosaic Company', type='stock'): 0.035, Asset(name='Caterpillar Inc.', type='stock'): 0.055, Asset(name='NextEra Energy', type='stock'): 0.1, Asset(name='Philip Morris International Inc.', type='stock'): 0.03, Asset(name='Tractor Supply Company', type='stock'): 0.035, Asset(name='GrainCorp Limited', type='stock'): 0.035, Asset(name='C3.ai', type='stock'): 0.05})

In [5]:
df_last_news = pd.read_sql(
    "SELECT * FROM vecs.sp500_news limit 100",
    engine,
)

In [6]:
df_last_news.head(2)

,id,metadata,content,embedding
0,8319246f-ad11-4836-95a4-069d8c7b9780,"{'tags': '['Kessler Topaz Meltzer & Check', 'L...",NET Investigation Reminder: Kessler Topaz Melt...,"[0.01551223,-0.017488744,0.016970761,-0.057686..."
1,2fccd1b3-4090-4950-b9a9-346904e215fe,"{'tags': '['Australia', 'Basic Materials', 'Bu...",US earnings: 22 Jul 2024 - America’s June 30 q...,"[-0.023108553,-0.02260535,0.007122234,-0.03734..."


In [8]:
newsletter_edition = newsletter_agent.create_newsletter(
    context=f"""
    Last week the porfolio has been created, so we have included the following assets, in the form (asset, weight):
    {str(current_portfolio.allocation)}

    The main news of the week are:
    {'  New: '.join(df_last_news["content"].tolist())}
    """
)

cache_path: C:\Users\Carlos\AppData\Roaming\adalflow\cache_OpenAIClient_gpt-4o-mini.db
Prompt:
______________________
<START_OF_SYSTEM_PROMPT>

You are a professional Newsletter creator that excell at narrative and catchy pieces.
Objective: Generate a high-quality weekly newsletter that includes updates, price movements, and portfolio operations.
Writing Guidelines:
- Tone and Style: Professional, informative, and engaging. Use a conversational tone to keep the reader interested.
- Language: Clear, concise, and free of jargon. Ensure complex terms are explained.
- Accuracy: Ensure all data and information are accurate and up-to-date.
- Visuals: Incorporate relevant visuals (charts, graphs, images) to enhance understanding.
- Length: Aim for a length of 1000-1500 words, ensuring content is comprehensive yet digestible.
- Utilize a optimistic tone for the narrative.
- Add the disclaimer about informational purposes only and does not constitute financial advice. 

You will be provider wit

In [9]:
newsletter: Newsletter = newsletter_edition.data


In [10]:
with open("./newsleter.txt", "+w") as io:
    io.write(newsletter.title)
    io.write(newsletter.body)